In [1]:
# Manage warning messages

import os, warnings
warnings.filterwarnings('ignore')

In [2]:
# Import necessary libraries

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
# Download and preprocess stock data as described in 5.1

df = yf.download('JPM', start='1980-01-01', end='2024-01-01')
df.index = pd.to_datetime(df.index)
df = df.asfreq('B')
df['Close'].interpolate(method='linear', inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Define the size of the training and testing set

train_size = int(len(df) * 0.8)
train_data = df['Close'][:train_size]
test_data = df['Close'][train_size:]


In [5]:
# Scale the data to the range [0, 1]

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(np.array(train_data).reshape(-1, 1))
scaled_test = scaler.transform(np.array(test_data).reshape(-1, 1))


In [6]:
# Function to create sequences of data for RNN input

def create_sequences(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)


In [7]:
time_step = 5  # Number of time steps to look back


In [8]:
# Create training sequences

X_train, y_train = create_sequences(scaled_train, time_step)
X_test, y_test = create_sequences(np.concatenate((scaled_train[-time_step:], scaled_test)), time_step)

In [ ]:
# Reshape the data for RNN input

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Build and compile the RNN model

model = Sequential()
model.add(SimpleRNN(50, input_shape=(time_step, 1)))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, batch_size=64, epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
143/143 [==============================] - 7s 13ms/step - loss: 0.0064 - val_loss: 0.6235
Epoch 2/25
143/143 [==============================] - 1s 7ms/step - loss: 5.4445e-04 - val_loss: 0.3493
Epoch 3/25
143/143 [==============================] - 1s 8ms/step - loss: 3.1096e-04 - val_loss: 0.1976
Epoch 4/25
143/143 [==============================] - 1s 10ms/step - loss: 2.4004e-04 - val_loss: 0.1371
Epoch 5/25
143/143 [==============================] - 1s 8ms/step - loss: 2.0609e-04 - val_loss: 0.1000
Epoch 6/25
143/143 [==============================] - 1s 8ms/step - loss: 1.9707e-04 - val_loss: 0.0852
Epoch 7/25
143/143 [==============================] - 1s 9ms/step - loss: 1.8496e-04 - val_loss: 0.0746
Epoch 8/25
143/143 [==============================] - 1s 7ms/step - loss: 1.8964e-04 - val_loss: 0.0654
Epoch 9/25
143/143 [==============================] - 1s 8ms/step - loss: 1.8344e-04 - val_loss: 0.0617
Epoch 10/25
143/143 [==============================] - 1s 10ms/ste

In [ ]:
# Make predictions on the training and test data

train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


72/72 [==============================] - 0s 2ms/step


In [ ]:
# Inverse transform the predictions to the original scale

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)


In [ ]:
# Fetch the actual stock prices for the forecast period

actual_2024 = yf.download('JPM', start='2024-01-02', end='2024-01-05')
actual_2024_prices = actual_2024['Close'].values

[*********************100%%**********************]  1 of 1 completed


In [ ]:
next = 3  # Number of days to forecast

In [ ]:
# Prepare the input for future predictions

future_inputs = df['Close'][-time_step:].values.reshape(1, time_step, 1)
future_predictions = []
for _ in range(next):
    future_pred = model.predict(future_inputs)
    future_predictions.append(future_pred[0, 0])
    future_inputs = np.append(future_inputs[:, 1:, :], future_pred.reshape(1, 1, 1), axis=1)

# Inverse transform the future predictions to the original scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))


1/1 [==============================] - 0s 22ms/step


In [ ]:
# Fetch the actual future stock prices for comparison

actual_future_df = yf.download('JPM', start='2024-01-02', end='2024-01-05')
actual_future_prices = actual_future_df['Close'].values


[*********************100%%**********************]  1 of 1 completed


In [ ]:
future_predictions

array([[236.50453],
       [157.90698],
       [114.85618]], dtype=float32)

In [ ]:
future_rmse = np.sqrt(mean_squared_error(actual_future_prices, future_predictions))
future_mape = mean_absolute_percentage_error(actual_future_prices, future_predictions)
print(f'Future RMSE: {future_rmse}')
print(f'Future MAPE: {future_mape}')


Future RMSE: 50.096682140595505
Future MAPE: 0.26088870955260085
